In [75]:
import glob
import numpy as np
import pandas as pd
import csv
import h5py
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Layer, Dense, Dropout, LSTM, GRU, Conv1D, Conv2D, Conv2DTranspose, MaxPooling2D, AveragePooling2D, UpSampling2D
from tensorflow.keras.layers import concatenate, GlobalMaxPooling1D, Flatten, BatchNormalization
from tensorflow.keras.layers import Activation, Reshape, TimeDistributed, Embedding, Input
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adamax, Adadelta, Adagrad, Nadam
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from tensorflow.keras.regularizers import l1, l2
from keras import backend as K
from sklearn.model_selection import KFold, StratifiedKFold, LeaveOneOut
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


from pycparser import parse_file as p
#from features import S0Gbg, S1Gbg, S2Gbg, S3Gbg, S4Gbg, S5Gbg, S6Gbg

from numpy.random import seed
from tensorflow.random import set_seed

from keras import backend as K

In [76]:
np.random.seed(0)
Normalise=0.005
tf.random.set_seed(0)

In [77]:
train = pd.read_csv(r'C:\Users\tejus\Desktop\Final yr project\Features\Features\Actual\HMM\MatMul_train_HMM_actual.csv')

In [78]:
train

,ACC,Kingdom,Partition,Membrane,Cytoplasm,Nucleus,Extracellular,Cell membrane,Mitochondrion,Plastid,...,Feat_391,Feat_392,Feat_393,Feat_394,Feat_395,Feat_396,Feat_397,Feat_398,Feat_399,Feat_400
0,Q28165,Metazoa,4,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.111411,0.244958,0.341650,0.217299,0.802411,0.559058,0.227327,0.288010,0.182961,1.723475
1,Q86U42,Metazoa,4,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.107259,0.262705,0.388817,0.218722,0.858188,0.587983,0.234405,0.282551,0.177832,1.880172
2,Q0GA42,Metazoa,3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.356197,0.633754,0.568381,0.466991,0.937301,1.178858,0.828762,1.238542,0.392853,4.291845
3,P82349,Metazoa,1,1.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.093193,0.184774,0.104548,0.178899,0.522350,0.314332,0.218914,0.270916,0.146021,1.486413
4,Q7L5N1,Metazoa,1,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.171199,0.313611,0.215853,0.317681,0.326752,0.463905,0.392336,0.531924,0.152525,1.729775
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28298,Q09712,Fungi,3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.436289,0.538304,0.690612,0.482105,0.727632,1.241338,0.729940,1.201819,0.904024,7.170935
28299,Q9V3C5,Metazoa,2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.754753,1.177238,1.066532,1.163425,1.529423,2.532042,1.670486,2.313325,1.660387,5.453517
28300,Q9SQQ9,Viridiplantae,4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.127730,0.246381,0.182832,0.391400,0.284849,0.468451,0.171616,0.176120,0.036062,2.257948
28301,Q9SQQ9-3,Viridiplantae,4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.125140,0.245346,0.193222,0.421375,0.285701,0.468638,0.185587,0.182496,0.035112,2.267903


In [79]:
X_train = train.drop(['ACC','Kingdom','Partition','Membrane','Cytoplasm','Nucleus','Extracellular','Cell membrane','Mitochondrion','Plastid','Endoplasmic reticulum','Lysosome/Vacuole','Golgi apparatus','Peroxisome','Sequence'],axis=1)

In [80]:
X_train

,Feat_1,Feat_2,Feat_3,Feat_4,Feat_5,Feat_6,Feat_7,Feat_8,Feat_9,Feat_10,...,Feat_391,Feat_392,Feat_393,Feat_394,Feat_395,Feat_396,Feat_397,Feat_398,Feat_399,Feat_400
0,14.982983,0.201766,1.303502,2.162008,0.533998,2.429846,0.343176,0.647653,1.350559,1.218167,...,0.111411,0.244958,0.341650,0.217299,0.802411,0.559058,0.227327,0.288010,0.182961,1.723475
1,14.216900,0.219421,1.194918,1.976242,0.550106,1.840085,0.336911,0.615057,1.287122,1.204021,...,0.107259,0.262705,0.388817,0.218722,0.858188,0.587983,0.234405,0.282551,0.177832,1.880172
2,15.555804,1.129827,3.171903,3.414553,2.083373,4.350188,1.036959,3.696123,2.462769,5.452501,...,0.356197,0.633754,0.568381,0.466991,0.937301,1.178858,0.828762,1.238542,0.392853,4.291845
3,9.442437,0.327668,0.655006,0.694246,0.384123,1.355365,0.274964,1.001960,0.726103,1.189358,...,0.093193,0.184774,0.104548,0.178899,0.522350,0.314332,0.218914,0.270916,0.146021,1.486413
4,8.554028,0.341592,1.241446,1.367293,0.692368,1.143770,0.502694,1.167590,1.023161,1.898444,...,0.171199,0.313611,0.215853,0.317681,0.326752,0.463905,0.392336,0.531924,0.152525,1.729775
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28298,5.481292,0.715920,0.618349,0.751800,2.638740,2.770277,0.530364,2.928688,0.977403,5.196495,...,0.436289,0.538304,0.690612,0.482105,0.727632,1.241338,0.729940,1.201819,0.904024,7.170935
28299,17.663655,2.106784,4.553804,4.790730,3.127160,6.314527,2.357921,4.745681,3.825695,8.191178,...,0.754753,1.177238,1.066532,1.163425,1.529423,2.532042,1.670486,2.313325,1.660387,5.453517
28300,4.889802,0.275934,0.829372,1.273311,0.398117,1.444185,0.415867,0.665779,1.122971,1.180808,...,0.127730,0.246381,0.182832,0.391400,0.284849,0.468451,0.171616,0.176120,0.036062,2.257948
28301,4.808752,0.267086,0.828082,1.274217,0.391082,1.431766,0.425447,0.667434,1.094252,1.177602,...,0.125140,0.245346,0.193222,0.421375,0.285701,0.468638,0.185587,0.182496,0.035112,2.267903


In [81]:
Y_train = train[['Membrane','Cytoplasm','Nucleus','Extracellular','Cell membrane','Mitochondrion','Plastid','Endoplasmic reticulum','Lysosome/Vacuole','Golgi apparatus','Peroxisome']]

In [82]:
Y_train

,Membrane,Cytoplasm,Nucleus,Extracellular,Cell membrane,Mitochondrion,Plastid,Endoplasmic reticulum,Lysosome/Vacuole,Golgi apparatus,Peroxisome
0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
28298,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
28299,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
28300,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28301,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [83]:
from sklearn.model_selection import train_test_split

Xtrain,Xvalidate,Ytrain,Yvalidate=train_test_split(X_train,Y_train,test_size=0.2,random_state=32)

In [84]:
Ytrain

,Membrane,Cytoplasm,Nucleus,Extracellular,Cell membrane,Mitochondrion,Plastid,Endoplasmic reticulum,Lysosome/Vacuole,Golgi apparatus,Peroxisome
2404,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26971,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11855,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14619,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21142,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
20414,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
27516,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9526,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9771,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [85]:
Xtrain.shape,Xvalidate.shape, Ytrain.shape,Yvalidate.shape

((22642, 400), (5661, 400), (22642, 11), (5661, 11))

In [86]:

    
hmm_input = Input(shape=(Xtrain.shape[1],), name='hmm_input')
d_output2 = Dense(400, activation='sigmoid')(hmm_input)
#d_output2 = Dropout(0.1)(d_output2) 
d_output2 = Dense(128, activation='relu')(d_output2)
#d_output2 = Dropout(0.1)(d_output2) 
d_output2 = Dense(64, activation='relu')(d_output2)
d_output2 = Dense(32, activation='relu')(d_output2)
# d_output2 = Dense(25, activation='sigmoid')(d_output2)
main_output = Dense(11, activation='softmax', name='main_output', kernel_regularizer=l2(0.01))(d_output2)

	

In [87]:

    
# hmm_input = Input(shape=(Xtrain.shape[1],), name='hmm_input')
# #d_output2 = Dense(400, activation='sigmoid')(hmm_input)

# d_output2 = Dense(100, activation='relu')(hmm_input)

# # d_output2 = Dense(25, activation='sigmoid')(d_output2)
# main_output = Dense(11, activation='softmax', name='main_output', kernel_regularizer=l2(0.01))(d_output2)

	

In [88]:
model = Model(inputs=[hmm_input], outputs=[main_output])
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

earlyStopping = EarlyStopping(monitor='val_accuracy', patience=20, verbose=0, mode='auto')

load_file = "78.h5"
checkpointer = ModelCheckpoint(monitor='accuracy', filepath=load_file, verbose=0, save_best_only=True)


	# history=model.fit({'bigram_input': X_biGram_Train}, {'main_output': Y_train}, epochs=500, batch_size=64, callbacks=[checkpointer, earlyStopping], verbose=0)

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hmm_input (InputLayer)      [(None, 400)]             0         
                                                                 
 dense_8 (Dense)             (None, 400)               160400    
                                                                 
 dense_9 (Dense)             (None, 128)               51328     
                                                                 
 dense_10 (Dense)            (None, 64)                8256      
                                                                 
 dense_11 (Dense)            (None, 32)                2080      
                                                                 
 main_output (Dense)         (None, 11)                363       
                                                                 


Total params: 222,427
Trainable params: 222,427
Non-trainable params: 0
_________________________________________________________________


In [89]:
with tf.device('/device:GPU:0'):
    
    history=model.fit({'hmm_input': Xtrain}, {'main_output': Ytrain},validation_data=(Xvalidate, Yvalidate), epochs=250,callbacks=[earlyStopping,checkpointer], batch_size=256)

Epoch 1/250
89/89 [==============================] - 2s 13ms/step - loss: 0.4837 - accuracy: 0.3310 - val_loss: 0.4124 - val_accuracy: 0.4533
Epoch 2/250
89/89 [==============================] - 1s 10ms/step - loss: 0.3814 - accuracy: 0.4790 - val_loss: 0.3523 - val_accuracy: 0.5095
Epoch 3/250
89/89 [==============================] - 1s 10ms/step - loss: 0.3347 - accuracy: 0.5101 - val_loss: 0.3194 - val_accuracy: 0.5427
Epoch 4/250
89/89 [==============================] - 1s 11ms/step - loss: 0.3094 - accuracy: 0.5306 - val_loss: 0.3004 - val_accuracy: 0.5443
Epoch 5/250
89/89 [==============================] - 1s 10ms/step - loss: 0.2944 - accuracy: 0.5388 - val_loss: 0.2872 - val_accuracy: 0.5600
Epoch 6/250
89/89 [==============================] - 1s 10ms/step - loss: 0.2842 - accuracy: 0.5488 - val_loss: 0.2814 - val_accuracy: 0.5635
Epoch 7/250
89/89 [==============================] - 1s 10ms/step - loss: 0.2759 - accuracy: 0.5580 - val_loss: 0.2731 - val_accuracy: 0.5739
Epoch 

In [90]:
Ytrain

,Membrane,Cytoplasm,Nucleus,Extracellular,Cell membrane,Mitochondrion,Plastid,Endoplasmic reticulum,Lysosome/Vacuole,Golgi apparatus,Peroxisome
2404,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26971,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11855,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14619,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21142,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
20414,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
27516,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9526,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9771,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [91]:
from ClassLabels import label_generator
from multiLabelMetrics import accuracy
for index, row in Xtrain.iterrows():
    single_row = Xtrain.loc[index].values.reshape(1, -1)

    # Make predictions
    predictions = model.predict(single_row)
    arr_Y=np.array([Ytrain.loc[index]])
    arr_pred = label_generator(predictions=predictions)
    test_acc = accuracy(arr_pred,arr_Y)
    
    x=test_acc.strictAccuracy()
    print(x)
    if x==100 :
        break

1/1 [==============================] - 0s 95ms/step
100.0


In [92]:
single_row = Xtrain.loc[index].values.reshape(1, -1)

# Make predictions
predictions = model.predict(single_row)
predictions

1/1 [==============================] - 0s 28ms/step


array([[1.9722439e-02, 1.3146888e-02, 9.4421607e-01, 3.8988395e-03,
        1.2724726e-02, 5.0205027e-04, 7.2129675e-05, 1.0972732e-03,
        2.5841740e-03, 2.0159408e-03, 1.9483306e-05]], dtype=float32)

In [93]:
from ClassLabels import label_generator
from multiLabelMetrics import accuracy
arr_Y=np.array([Ytrain.loc[index]])
arr_pred = label_generator(predictions=predictions)
arr_pred

array([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [94]:
arr_Y

array([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [95]:
test_acc = accuracy(arr_pred,arr_Y)
test_acc.strictAccuracy()

100.0

In [96]:
test_acc.relaxedAccuracy()

100.0

In [97]:
model.load_weights(load_file)
score = model.evaluate({'hmm_input': Xtrain},{'main_output': Ytrain})

708/708 [==============================] - 8s 11ms/step - loss: 0.1995 - accuracy: 0.6880


In [98]:
model.load_weights(load_file)
score = model.evaluate({'hmm_input': Xvalidate},{'main_output': Yvalidate})

177/177 [==============================] - 1s 5ms/step - loss: 0.2307 - accuracy: 0.6368


In [99]:

pred_scores = model.predict(Xtrain)
arr_Y = np.array(Ytrain)
arr_pred = label_generator(predictions=pred_scores)
acc = accuracy(arr_pred,arr_Y)
acc.strictAccuracy()

708/708 [==============================] - 2s 3ms/step


38.538998321703026

In [100]:
acc.relaxedAccuracy()

91.05042118026834

In [101]:

pred_scores = model.predict(Xvalidate)
arr_Y = np.array(Yvalidate)
arr_pred = label_generator(predictions=pred_scores)

177/177 [==============================] - 1s 4ms/step


In [102]:

acc = accuracy(arr_pred,arr_Y)
acc.strictAccuracy()

34.56986398162869

In [103]:
acc.relaxedAccuracy()

90.04994299111945

In [104]:

test = pd.read_csv(r'C:\Users\tejus\Desktop\Final yr project\Features\Features\Actual\HMM\MatMul_test_HMM_actual.csv')

In [105]:
test

,sid,Cell membrane,Cytoplasm,Endoplasmic reticulum,Golgi apparatus,Lysosome/Vacuole,Mitochondrion,Nucleus,Peroxisome,Lengths,...,Feat_391,Feat_392,Feat_393,Feat_394,Feat_395,Feat_396,Feat_397,Feat_398,Feat_399,Feat_400
0,ENSP00000355743,0,0,0,0,0,0,1,0,496,...,0.258011,0.353851,0.435284,0.413877,0.594366,0.595805,0.396866,0.544421,0.424152,6.692077
1,ENSP00000429628,0,0,0,0,0,0,1,0,173,...,0.093635,0.185587,0.206210,0.225075,0.214002,0.339029,0.246747,0.250573,0.089538,0.442268
2,ENSP00000266544,0,0,0,0,0,1,0,0,377,...,0.278993,0.339251,0.438263,0.351402,0.544947,0.618431,0.499604,0.843471,0.315650,1.232763
3,ENSP00000470652,0,0,0,1,0,0,0,0,410,...,0.363690,0.433065,0.379793,0.382677,0.473912,0.860343,0.664466,0.945804,0.395950,0.997238
4,ENSP00000460751,0,0,0,0,0,0,1,0,163,...,0.037747,0.119094,0.092005,0.155926,0.215646,0.201815,0.094544,0.163751,0.268752,2.492428
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1680,ENSP00000379284,0,0,0,0,0,1,0,0,372,...,0.302538,0.348803,0.334693,0.324379,0.641153,0.535000,0.460455,0.666990,0.304205,1.033432
1681,ENSP00000430164,0,1,0,0,0,0,0,0,73,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
1682,ENSP00000414655,0,1,0,0,0,0,0,0,174,...,0.014237,0.062561,0.143737,0.114862,0.048551,0.241509,0.102259,0.037878,0.008699,1.638261
1683,ENSP00000392070,0,0,0,0,0,0,1,0,327,...,0.184005,0.214807,0.337576,0.290199,0.442781,0.461941,0.351031,0.541925,0.135670,1.037373


In [106]:
test.dropna(inplace=True)

In [107]:
Xtest=test.drop(['sid','Cell membrane','Cytoplasm','Endoplasmic reticulum','Golgi apparatus','Lysosome/Vacuole','Mitochondrion','Nucleus','Peroxisome','Lengths','fasta'],axis=1)

In [108]:
Xtest

,Feat_1,Feat_2,Feat_3,Feat_4,Feat_5,Feat_6,Feat_7,Feat_8,Feat_9,Feat_10,...,Feat_391,Feat_392,Feat_393,Feat_394,Feat_395,Feat_396,Feat_397,Feat_398,Feat_399,Feat_400
0,10.511840,0.681454,1.467145,2.096745,0.502221,1.417815,0.453600,1.072239,1.948673,1.454342,...,0.258011,0.353851,0.435284,0.413877,0.594366,0.595805,0.396866,0.544421,0.424152,6.692077
1,1.657306,0.311132,0.530636,0.704214,0.324607,0.519876,0.353429,0.413341,0.636022,0.694782,...,0.093635,0.185587,0.206210,0.225075,0.214002,0.339029,0.246747,0.250573,0.089538,0.442268
2,7.251676,0.515472,1.437248,1.764437,1.077316,2.332655,0.671138,1.967690,1.592020,3.348442,...,0.278993,0.339251,0.438263,0.351402,0.544947,0.618431,0.499604,0.843471,0.315650,1.232763
3,6.283611,0.732520,0.979360,1.151415,2.139442,3.365095,0.477635,2.617250,1.094336,4.500879,...,0.363690,0.433065,0.379793,0.382677,0.473912,0.860343,0.664466,0.945804,0.395950,0.997238
4,5.599081,0.242164,0.420853,0.709060,0.202405,0.316280,0.257576,0.420584,0.380964,0.757186,...,0.037747,0.119094,0.092005,0.155926,0.215646,0.201815,0.094544,0.163751,0.268752,2.492428
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1680,3.287648,0.363539,1.261358,1.569617,1.170052,1.563395,0.633403,1.594907,1.355015,2.561954,...,0.302538,0.348803,0.334693,0.324379,0.641153,0.535000,0.460455,0.666990,0.304205,1.033432
1681,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
1682,3.704577,0.121874,0.358170,0.334261,0.048148,0.377475,0.187446,0.215600,0.446949,0.381701,...,0.014237,0.062561,0.143737,0.114862,0.048551,0.241509,0.102259,0.037878,0.008699,1.638261
1683,4.817369,0.435462,1.173849,1.843065,0.896056,1.756875,0.508191,1.135122,1.224170,2.156034,...,0.184005,0.214807,0.337576,0.290199,0.442781,0.461941,0.351031,0.541925,0.135670,1.037373


In [109]:
Ytest=test[['Cytoplasm','Nucleus','Cell membrane','Mitochondrion','Endoplasmic reticulum','Lysosome/Vacuole','Golgi apparatus','Peroxisome']]

In [110]:
Ytest

,Cytoplasm,Nucleus,Cell membrane,Mitochondrion,Endoplasmic reticulum,Lysosome/Vacuole,Golgi apparatus,Peroxisome
0,0,1,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,1,0
4,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
1680,0,0,0,1,0,0,0,0
1681,1,0,0,0,0,0,0,0
1682,1,0,0,0,0,0,0,0
1683,0,1,0,0,0,0,0,0


In [111]:
pred_scores = model.predict(Xtest)
arr_Y = np.array(Ytest)
arr_pred = label_generator(predictions=pred_scores)
print(arr_Y.shape)
print(arr_pred.shape)

53/53 [==============================] - 0s 3ms/step
(1685, 8)
(1685, 11)


In [112]:
df_arr_Y = pd.DataFrame(arr_Y)
df_arr_pred = pd.DataFrame(arr_pred)

df_arr_pred.drop(df_arr_pred.columns[[0,3,6]], axis = 1, inplace = True)

arr_Y = df_arr_Y.values
arr_pred = df_arr_pred.values

In [113]:
print(arr_Y.shape)
print(arr_pred.shape)

(1685, 8)
(1685, 8)


In [114]:
acc = accuracy(arr_pred,arr_Y)
acc.strictAccuracy()

30.68249258160237

In [115]:
acc.relaxedAccuracy()

86.28338278931751